<center>
<img src="https://habrastorage.org/files/fd4/502/43d/fd450243dd604b81b9713213a247aa20.jpg">
    
## [mlcourse.ai](https://mlcourse.ai) - Open Machine Learning Course
Auteur: [Yury Kashnitskiy](https://yorko.github.io) (@yorko). Edité et traduit par [Ousmane Cissé](https://github.com/oussou-dev). Ce matériel est soumis aux termes et conditions de la licence [Creative Commons CC BY-NC-SA 4.0](https://creativecommons.org/licenses/by-nc-sa/4.0/). L'utilisation gratuite est autorisée à des fins non commerciales.</center>

## <center> Mission 4. Détection des sarcasmes avec la régression logistique. Solution
    
Nous utiliserons le jeu de données de [paper](https://arxiv.org/abs/1704.05579) "Un grand corpus auto-annoté pour le sarcasme" avec plus de 1 million de commentaires provenant du site Reddit, étiquetés comme sarcastiques ou non. Une version complète peut être trouvée sur Kaggle sous la forme d'un [Jeu de données Kaggle](https://www.kaggle.com/danofer/sarcasm).</center>

In [ ]:
PATH_TO_DATA = "../input/sarcasm/train-balanced-sarcasm.csv"

In [ ]:
# some necessary imports
import os

import numpy as np
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline

In [ ]:
train_df = pd.read_csv(PATH_TO_DATA)

In [ ]:
train_df.head()

In [ ]:
train_df.info()

Certains commentaires sont manquants, alors nous supprimons les lignes correspondantes.

In [ ]:
train_df.dropna(subset=["comment"], inplace=True)

Nous remarquons que le jeu de données est bien équilibré

In [ ]:
train_df["label"].value_counts()

Nous divisons les données en données d'entraînement et données de validation.

In [ ]:
train_texts, valid_texts, y_train, y_valid = train_test_split(
    train_df["comment"], train_df["label"], random_state=17
)

## Les tâches:
1. Analysez le jeu de données, faites des graphiques. Ce [kernel](https://www.kaggle.com/sudalairajkumar/simple-exploration-notebook-qiqc) pourrait servir d'exemple.
2. Créez un Tf-Idf + un pipeline de régression logistique pour prédire le sarcasme (`label`) en fonction du texte d'un commentaire sur Reddit (` comment`).
3. Tracer les mots/bigrammes qui sont les plus prédictifs du sarcasme (vous pouvez utiliser [eli5](https://github.com/TeamHG-Memex/eli5) pour cela)
4. (facultatif) ajouter des sous-titres (subreddits) en tant que nouvelles caractéristiques pour améliorer les performances du modèle. Appliquez ici l’approche Sac de mots (Bag of Words), c’est-à-dire traitez chaque sous-titre comme une nouvelle caractéristique.

### Partie 1. Analyse exploratoire des données

La distribution des longueurs pour les commentaires sarcastiques et normaux est presque la même.

In [ ]:
train_df.loc[train_df["label"] == 1, "comment"].str.len().apply(np.log1p).hist(
    label="sarcastic", alpha=0.5
)
train_df.loc[train_df["label"] == 0, "comment"].str.len().apply(np.log1p).hist(
    label="normal", alpha=0.5
)
plt.legend();

In [ ]:
from wordcloud import STOPWORDS, WordCloud

In [ ]:
wordcloud = WordCloud(
    background_color="black",
    stopwords=STOPWORDS,
    max_words=200,
    max_font_size=100,
    random_state=17,
    width=800,
    height=400,
)

Le nuage de mots est beau, mais pas très utile

In [ ]:
plt.figure(figsize=(16, 12))
wordcloud.generate(str(train_df.loc[train_df["label"] == 1, "comment"]))
plt.imshow(wordcloud);

In [ ]:
plt.figure(figsize=(16, 12))
wordcloud.generate(str(train_df.loc[train_df["label"] == 0, "comment"]))
plt.imshow(wordcloud);

Analysons si certains sous-titres (subreddits) sont plus "sarcastiques" en moyenne que d'autres

In [ ]:
sub_df = train_df.groupby("subreddit")["label"].agg([np.size, np.mean, np.sum])
sub_df.sort_values(by="sum", ascending=False).head(10)

In [ ]:
sub_df[sub_df["size"] > 1000].sort_values(by="mean", ascending=False).head(10)

La même chose pour les auteurs ne donne pas beaucoup de perspicacité. Hormis le fait que les commentaires de quelqu'un aient été échantillonnés, nous pouvons constater le même nombre de commentaires sarcastiques et non sarcastiques.

In [ ]:
sub_df = train_df.groupby("author")["label"].agg([np.size, np.mean, np.sum])
sub_df[sub_df["size"] > 300].sort_values(by="mean", ascending=False).head(10)

In [ ]:
sub_df = (
    train_df[train_df["score"] >= 0]
    .groupby("score")["label"]
    .agg([np.size, np.mean, np.sum])
)
sub_df[sub_df["size"] > 300].sort_values(by="mean", ascending=False).head(10)

In [ ]:
sub_df = (
    train_df[train_df["score"] < 0]
    .groupby("score")["label"]
    .agg([np.size, np.mean, np.sum])
)
sub_df[sub_df["size"] > 300].sort_values(by="mean", ascending=False).head(10)

### Partie 2. Entraîner le modèle

In [ ]:
# build bigrams, put a limit on maximal number of features
# and minimal word frequency
tf_idf = TfidfVectorizer(ngram_range=(1, 2), max_features=50000, min_df=2)
# multinomial logistic regression a.k.a softmax classifier
logit = LogisticRegression(C=1, n_jobs=4, solver="lbfgs", random_state=17, verbose=1)
# sklearn's pipeline
tfidf_logit_pipeline = Pipeline([("tf_idf", tf_idf), ("logit", logit)])

In [ ]:
%%time
tfidf_logit_pipeline.fit(train_texts, y_train)

In [ ]:
%%time
valid_pred = tfidf_logit_pipeline.predict(valid_texts)

In [ ]:
accuracy_score(y_valid, valid_pred)

### Partie 3. Expliquer le modèle

In [ ]:
def plot_confusion_matrix(
    actual,
    predicted,
    classes,
    normalize=False,
    title="Confusion matrix",
    figsize=(7, 7),
    cmap=plt.cm.Blues,
    path_to_save_fig=None,
):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    import itertools

    from sklearn.metrics import confusion_matrix

    cm = confusion_matrix(actual, predicted).T
    if normalize:
        cm = cm.astype("float") / cm.sum(axis=1)[:, np.newaxis]

    plt.figure(figsize=figsize)
    plt.imshow(cm, interpolation="nearest", cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=90)
    plt.yticks(tick_marks, classes)

    fmt = ".2f" if normalize else "d"
    thresh = cm.max() / 2.0
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(
            j,
            i,
            format(cm[i, j], fmt),
            horizontalalignment="center",
            color="white" if cm[i, j] > thresh else "black",
        )

    plt.tight_layout()
    plt.ylabel("Predicted label")
    plt.xlabel("True label")

    if path_to_save_fig:
        plt.savefig(path_to_save_fig, dpi=300, bbox_inches="tight")

La matrice de confusion est assez équilibrée.

In [ ]:
plot_confusion_matrix(
    y_valid,
    valid_pred,
    tfidf_logit_pipeline.named_steps["logit"].classes_,
    figsize=(8, 8),
)

En effet, nous pouvons reconnaître certaines expressions indiquant un sarcasme. Comme "oui bien sûr".

In [ ]:
import eli5

eli5.show_weights(
    estimator=tfidf_logit_pipeline.named_steps["logit"],
    vec=tfidf_logit_pipeline.named_steps["tf_idf"],
)

So sarcasm detection is easy.
<img src="https://habrastorage.org/webt/1f/0d/ta/1f0dtavsd14ncf17gbsy1cvoga4.jpeg">

### Partie 4. Améliorer le modèle

In [ ]:
subreddits = train_df["subreddit"]
train_subreddits, valid_subreddits = train_test_split(subreddits, random_state=17)

Nous aurons des vectoriseurs Tf-Idf séparés pour les commentaires et les sous-titres (subreddits). Il est également possible de s'en tenir à un pipeline, mais dans ce cas, cela devient un peu moins simple. [Exemple](https://stackoverflow.com/questions/36731813/computing-separate-tfidf-scores-for-two-dwifferent-columns-using-sklearn)

In [ ]:
tf_idf_texts = TfidfVectorizer(ngram_range=(1, 2), max_features=50000, min_df=2)
tf_idf_subreddits = TfidfVectorizer(ngram_range=(1, 1))

Faites des transformations séparément pour les commentaires et les sous-titres.

In [ ]:
%%time
X_train_texts = tf_idf_texts.fit_transform(train_texts)
X_valid_texts = tf_idf_texts.transform(valid_texts)

In [ ]:
X_train_texts.shape, X_valid_texts.shape

In [ ]:
%%time
X_train_subreddits = tf_idf_subreddits.fit_transform(train_subreddits)
X_valid_subreddits = tf_idf_subreddits.transform(valid_subreddits)

In [ ]:
X_train_subreddits.shape, X_valid_subreddits.shape

Ensuite, empilez toutes les caractéristiques ensemble.

In [ ]:
from scipy.sparse import hstack

X_train = hstack([X_train_texts, X_train_subreddits])
X_valid = hstack([X_valid_texts, X_valid_subreddits])

In [ ]:
X_train.shape, X_valid.shape

Entraînez de même la régression logistique.

In [ ]:
logit.fit(X_train, y_train)

In [ ]:
%%time
valid_pred = logit.predict(X_valid)

In [ ]:
accuracy_score(y_valid, valid_pred)

Comme on peut le constater, la précision a légèrement augmenté.

## Liens:
  - Machine learning library [Scikit-learn](https://scikit-learn.org/stable/index.html) (a.k.a. sklearn)
  - Kernels on [logistic regression](https://www.kaggle.com/kashnitsky/topic-4-linear-models-part-2-classification) and its applications to [text classification](https://www.kaggle.com/kashnitsky/topic-4-linear-models-part-4-more-of-logit), also a [Kernel](https://www.kaggle.com/kashnitsky/topic-6-feature-engineering-and-feature-selection) on feature engineering and feature selection
  - [Kaggle Kernel](https://www.kaggle.com/abhishek/approaching-almost-any-nlp-problem-on-kaggle) "Approaching (Almost) Any NLP Problem on Kaggle"
  - [ELI5](https://github.com/TeamHG-Memex/eli5) to explain model predictions